# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import cassandra
import re
import os
import glob
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/Users/felipe/Projects/udacity/data-engineering-nanodegree/1b_data-modeling-cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
# print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data looks like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code

#### Creating a Cluster

In [5]:
# Make a connection to a Cassandra instance in my local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace "sparkifydb"

In [6]:
session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkifydb
                   WITH REPLICATION =
                   {'class': 'SimpleStrategy', 'replication_factor': 1}""")

#### Set Keyspace to "sparkifydb"

In [7]:
session.set_keyspace('sparkifydb')

## Queries to answer the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




#### First, drop all tables in case they already exist

In [8]:
drop_session_history = 'DROP TABLE IF EXISTS session_history'
drop_user_history = 'DROP TABLE IF EXISTS user_history'
drop_song_history = 'DROP TABLE IF EXISTS song_history'

drop_table_queries = [drop_session_history, drop_user_history, drop_song_history]
for query in drop_table_queries:
    session.execute(query)

#### Now let us create tables to answer the 3 questions above. I am using the "one table per query" strategy.

Table to answer "1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4"
> **Explanation**: this table must allow for filtering by sessionId and then by itemInSession. Therefore, we use those as partition key and clustering column, respectively. The reamining table columns are the values one wish to retrieve in order to answer the question.

In [9]:
create_session_history = """CREATE TABLE IF NOT EXISTS session_history (session_id int,
                                                                        item_in_session int,
                                                                        artist text,
                                                                        song_title text,
                                                                        song_length double,
                                                                        PRIMARY KEY(session_id, item_in_session))"""

Table to answer "2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182"
> **Explanation**: we choose userId as the partition key, since it is the first filter we wish to apply; userId as the first clustering column because later we need to filter by its value; and itemInSession as the second clustering column to be sure the query results will be sorted by its value. The remaining columns are the values one wish to retrieve in order to answer the question.

In [10]:
create_user_history = """CREATE TABLE IF NOT EXISTS user_history (user_id int,
                                                                  session_id int,
                                                                  item_in_session int,
                                                                  artist text,
                                                                  song_title text,
                                                                  first_name text,
                                                                  last_name text,
                                                                  PRIMARY KEY(user_id, session_id, item_in_session))"""

Table to answer "3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'"
> **Explanation**: the song's title must be the partition key because we only want to filter by its value. However, we need userId as a clustering column, otherwise different records of the same song with different userId's would overwrite each other — and we would end up with only one userId per song in the final table. The reamining columns are the ones we want to retrieve in order to answer the question.

In [11]:
create_song_history = """CREATE TABLE IF NOT EXISTS song_history (song_title text,
                                                                  user_id int,
                                                                  first_name text,
                                                                  last_name text,
                                                                  PRIMARY KEY(song_title, user_id))"""

In [12]:
# Now create all tables
create_table_queries = [create_session_history, create_user_history, create_song_history]
for query in create_table_queries:
    session.execute(query)

#### Insert data from `event_datafile_new.csv` into all tables. I am doing so by first creating a stream of events and then processing each event into the 3 tables.

In [13]:
def stream_from_file(csv_file_path):
    """
    Iterates over lines of CSV file located in csv_file_path until all events have been consumed.

    :param csv_file_path: path to CSV file containing all events
    :return: iterator of dict with event data
    """
    with open(csv_file_path, 'r', encoding='utf8', newline='') as csv_file:
        csv_reader = csv.reader(csv_file)
        header = next(csv_reader)
        for line in csv_reader:
            if line[0] == '':
                continue
            event = {k: v for k, v in zip(header, line)}
            event['length'] = float(event['length'])
            event['userId'] = int(event['userId'])
            event['itemInSession'] = int(event['itemInSession'])
            event['sessionId'] = int(event['sessionId'])
            yield event

In [14]:
# Insert statement for table that answers "1. Give me the artist, song title and song's length in the music app 
#  history that was heard during sessionId = 338, and itemInSession = 4"
insert_session_history = """INSERT INTO session_history (session_id, item_in_session, artist, song_title, song_length)
                            VALUES (%s, %s, %s, %s, %s)"""

# Insert statement for table that answers "2. Give me only the following: name of artist, song (sorted by itemInSession) 
#  and user (first and last name) for userid = 10, sessionid = 182"
insert_user_history = """INSERT INTO user_history (user_id, session_id, item_in_session,
                                                   artist, song_title, first_name, last_name)
                         VALUES (%s, %s, %s, %s, %s, %s, %s)"""

# Insert statement for table that answers "3. Give me every user name (first and last) in my music app history who 
#  listened to the song 'All Hands Against His Own'"
insert_song_history = """INSERT INTO song_history (song_title, user_id, first_name, last_name)
                         VALUES (%s, %s, %s, %s)"""

In [15]:
# Create event stream and insert event data into all tables
event_stream = stream_from_file('event_datafile_new.csv')
for i, event in enumerate(event_stream, start=1):
    print(f'Processing event {i}.', end='\r', flush=True)
    session.execute(insert_session_history, (event['sessionId'], event['itemInSession'], event['artist'],
                                             event['song'], event['length']))
    session.execute(insert_user_history, (event['userId'], event['sessionId'], event['itemInSession'],
                                          event['artist'], event['song'], event['firstName'], event['lastName']))
    session.execute(insert_song_history, (event['song'], event['userId'], event['firstName'], event['lastName']))
print('All events were inserted.')

All events were inserted.


#### Now we can finally answer the questions.

In [16]:
def process_query(session, query, values):
    """
    Utility function to process cassandra queries and print its results.
    
    :param session: an open cassandra session
    :param query: a string representing a CQL query; it allows placeholders to be represented as %s
    :param values: list of values to be inserted in "%s" placeholders of query
    """
    rows = session.execute(query, values)
    for row in rows:
        print(row)

### 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [17]:
select_session_data = """SELECT artist, song_title, song_length
                         FROM session_history WHERE session_id=%s AND item_in_session=%s"""
process_query(session, select_session_data, (338, 4))

Row(artist='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.3073)


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [18]:
select_user_data = """SELECT artist, song_title, first_name, last_name
                      FROM user_history WHERE user_id=%s AND session_id=%s"""
process_query(session, select_user_data, (10, 182))

Row(artist='Down To The Bone', song_title="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist='Three Drives', song_title='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist='Sebastien Tellier', song_title='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [19]:
select_song_data = """SELECT first_name, last_name
                      FROM song_history WHERE song_title=%s"""
process_query(session, select_song_data, ('All Hands Against His Own',))

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')
Row(first_name='Sara', last_name='Johnson')


### Job's done. Let us drop all tables and close the connection

In [20]:
for query in drop_table_queries:
    session.execute(query)

In [21]:
session.shutdown()
cluster.shutdown()